In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

print(torch.__version__)
print(torchvision.__version__)

In [ ]:
device="cuda" if torch.cuda.is_available() else"cpu"
device

In [ ]:
train_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
    target_transform=None
)

test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
    target_transform=None
)

In [ ]:
len(train_data),len(test_data)

In [ ]:
class_name=train_data.classes
class_name

In [ ]:
class_to_idx=train_data.class_to_idx
class_to_idx

In [ ]:
train_data.data.shape

In [ ]:
image, label=train_data[0]
print(f"image shape{image.shape}")
plt.imshow(image.squeeze())
plt.title(label)

In [ ]:
plt.imshow(image.squeeze(),cmap="gray")
plt.title(class_name[label])
plt.axis(False
        )

In [ ]:
#plot more images random
torch.manual_seed(42)
fig=plt.figure(figsize=(8,8))
rows,cols=4,4
for i in range (1,rows*cols+1):
  random_idx=torch.randint(0,len(train_data),size=[1]).item()
  img,label=train_data[random_idx]
  fig.add_subplot(rows,cols,i)
  plt.imshow(img.squeeze(),cmap="gray")
  plt.title(class_name[label])
  plt.axis(False)


In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE=32

train_dataloader=DataLoader(dataset=train_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True)

test_dataloader=DataLoader(dataset=test_data,
                           batch_size=BATCH_SIZE,
                           shuffle=False)

train_dataloader,test_dataloader

In [ ]:
len(train_dataloader),len(test_dataloader)

In [ ]:
train_features_batch, train_labels_batch=next(iter(train_dataloader))
train_features_batch

In [ ]:
torch.manual_seed(42)
random_idx=torch.randint(0,len(train_features_batch),size=[1]).item()
rand_img,rand_label=train_features_batch[random_idx],train_labels_batch[random_idx]
plt.imshow(rand_img.squeeze(),cmap="gray")
plt.title(class_name[rand_label])
plt.axis(False);
print(f"image size:{rand_img.shape} ")
print(f"Label:{rand_label} label size:{rand_label.shape}")


In [ ]:
flatten_model=nn.Flatten()

X=train_features_batch[0]

output=flatten_model(X)

print(f"x shape before flattening:{X.shape}")
print(f"x shape after flattening:{output.shape}")

In [ ]:
from torch import nn
class FashionMNISTModel0(nn.Module):
  def __init__(self,input_shape:int,
              hidden_units:int,
               output_shape:int):
    super().__init__()

    self.layer_stacks=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape,out_features=hidden_units),
        nn.Linear(in_features=hidden_units,out_features=output_shape)
    )

  def forward(self,x):
    return self.layer_stacks(x)


In [ ]:
torch.manual_seed(42)
model_0=FashionMNISTModel0(input_shape=28*28,
                           hidden_units=10,
                           output_shape=10).to(device)

model_0

In [ ]:
model_0.state_dict()

In [ ]:
import requests
from pathlib import Path

if Path("helper_funstions.py").is_file():
  print("file already downloaded, skipping")
else:
  print("downloading helper_funcitons.py")
  request=requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/refs/heads/main/helper_functions.py")
  with open("helper_functions.py","wb") as f:
    f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn

loss_fn=nn.CrossEntropyLoss()

optimizer=torch.optim.SGD(params=model_0.parameters(),lr=0.01)

In [ ]:
from timeit import default_timer as timer

def print_train_time(start:float,
                     end:float,
                     device: torch.device = None ):
  """prints the difference between start and end time"""

  total_time=end-start
  print(f"train time on:{device}:{total_time:.3f}seconds")
  return total_time

In [ ]:
start_time=timer()

end_time=timer()

print_train_time(start=start_time,end=end_time)

In [ ]:
from tqdm.auto import tqdm

torch.manual_seed(42)

train_time_start=timer()
epochs=4



for epoch in tqdm(range(epochs)):
  print(f"Epoch:{epoch}")
  train_loss=0

  for batch,(X,y) in enumerate(train_dataloader):

    X,y=X.to(device),y.to(device)
    model_0.train()

    y_pred=model_0(X)
    loss=loss_fn(y_pred,y)
    train_loss+=loss

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if batch%350==0:
      print(f"looked at{batch * len(X)}/{len(train_dataloader.dataset)} samples")

  train_loss=train_loss/len(train_dataloader)


  test_loss,test_acc=0,0
  model_0.eval()
  with torch.inference_mode():
    for X_test,y_test in test_dataloader:
      X_test,y_test=X_test.to(device),y_test.to(device)
      test_pred=model_0(X_test)

      test_loss+=loss_fn(test_pred,y_test)

      test_acc+= accuracy_fn(y_true=y_test,y_pred=test_pred.argmax(dim=1))

    test_loss/=len(test_dataloader)

    test_acc/=len(test_dataloader)

  print(f"train Loss:{train_loss:.4f} | test Loss:{test_loss:.4f}|test accuracy:{test_acc:.4f}")

  train_time_end=timer()

  total_train_time_model_0=print_train_time(start=train_time_start,end=train_time_end,
                                            device=str(next(model_0.parameters()).device))


In [ ]:
torch.manual_seed(42)

def eval_model(model:torch.nn.Module,
               data_loader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               accuracy_fn):
  """returns a dictionary of model predicting on dataloader"""

  loss,acc=0,0
  model.eval()
  with torch.inference_mode():
    for X,y in data_loader:
      X,y=X.to(device),y.to(device)
      y_pred=model(X)



      loss+=loss_fn(y_pred,y)

      acc+=accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

    loss/=len(data_loader)
    acc/=len(data_loader)
  return{"model_name":model.__class__.__name__,
         "model_loss":loss.item(),
         "model_accuracy":acc}

In [ ]:
model_0_results=eval_model(model_0,test_dataloader,loss_fn,accuracy_fn)

In [ ]:
print(model_0_results)

In [ ]:
torch.manual_seed(42)

class FashionMNISTModel1(nn.Module):
  def __init__(self,input_shape:int,
               hidden_units:int,
               output_shape:int):
    super().__init__()
    self.layer_stacks=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape,out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units,out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units,out_features=output_shape)
    )

  def forward(self,x):
    return self.layer_stacks(x)

In [ ]:
class_name

In [ ]:
torch.manual_seed(42)

model_1=FashionMNISTModel1(input_shape=28*28,
                           hidden_units=10,
                           output_shape=len(class_name)).to(device)

In [ ]:
from helper_functions import accuracy_fn

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=model_1.parameters(),
                          lr=0.01)

In [ ]:
def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               accuracy_fn,
               device:torch.device=device):
  torch.manual_seed(42)


  model.train()
  train_loss,train_accuracy=0,0
  for batch, (X, y) in enumerate(dataloader):
    X,y=X.to(device),y.to(device)

    y_pred=model(X)

    loss=loss_fn(y_pred,y)
    train_loss+=loss
    train_accuracy+=accuracy_fn(y_true=y,y_pred=y_pred.argmax(dim=1))
    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if(batch%350==0):
      print(f"looked at :{batch*len(X)}/{len(dataloader.dataset)}")
  train_loss/=len(dataloader)
  train_accuracy/=len(dataloader)
  print(f"Train Loss:{train_loss:.4f}, train accuracy:{train_accuracy:.4f}")

In [ ]:
def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module,
              accuracy_fn,
              device:torch.device=device
              ):
  model.eval()
  test_loss,test_acc=0,0
  with torch.inference_mode():
    for X,y in dataloader:
      X,y=X.to(device),y.to(device)
      test_pred=model(X)

      loss=loss_fn(test_pred,y)

      test_loss+=loss

      test_acc+=accuracy_fn(y_true=y,y_pred=test_pred.argmax(dim=1))
    test_loss/=len(dataloader)
    test_acc/=len(dataloader)
    print(f"test loss:{test_loss:.4f} | test accuracy:{test_acc:.4f}")



In [ ]:
torch.manual_seed(42)

from timeit import default_timer as timer

train_timer_start_1=timer()
epochs=4

for epoch in tqdm(range(epochs)):
  print("Epoch:",epoch)
  train_step(model=model_1,dataloader=train_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn,optimizer=optimizer,device=device)

  test_step(model=model_1,dataloader=test_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn,device=device)

train_timer_stop_1=timer()
total_train_time_model_1=print_train_time(start=train_timer_start_1,
                                          end=train_timer_stop_1
                                          )

In [ ]:
model_0_results

In [ ]:
model_1_results=eval_model(model=model_1,data_loader=test_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn)
model_1_results

In [ ]:
class FashionMNISTModel2(nn.Module):
  def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
    super().__init__()
    self.conv_block1=nn.Sequential(
        nn.Conv2d(in_channels=input_shape,out_channels=hidden_units,
                  kernel_size=3,padding=1,stride=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,
                  kernel_size=3,padding=1,stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
        )

    self.conv_block2=nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,
                  kernel_size=3,padding=1,stride=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,out_channels=output_shape,
                  kernel_size=3,padding=1,stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*7*7,out_features=output_shape)
    )

  def forward(self,x):
    x=self.conv_block1(x)
    #print(x.shape)
    x=self.conv_block2(x)
    #print(x.shape)
    x=self.classifier(x)
    return x



In [ ]:
torch.manual_seed(42)
model_2=FashionMNISTModel2(input_shape=1,hidden_units=10,output_shape=len(class_name)).to(device)
model_2

In [ ]:
torch.manual_seed(42)
images=torch.rand(32,3,64,64)
image_0=images[0]
print("images shape:",images.shape)
print("single image shape",image_0.shape)
print("single image:",image_0)

In [ ]:
torch.manual_seed(42)
conv_layer=nn.Conv2d(in_channels=3,out_channels=10,
                     kernel_size=3,
                     padding=0,
                     stride=1)

conv_output=conv_layer(image_0)
conv_output.shape

In [ ]:
from helper_functions import accuracy_fn
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=model_2.parameters(),
                          lr=0.01)

In [ ]:
torch.manual_seed(42)

from timeit import default_timer as timer
train_time_start_m2=timer()

epochs=7

for epoch in tqdm(range(epochs)):
  print("EPoch:",epoch)
  train_step(model=model_2,dataloader=train_dataloader,loss_fn=loss_fn,optimizer=optimizer,device=device,accuracy_fn=accuracy_fn)

  test_step(model=model_2,dataloader=test_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn,device=device)

train_time_end_m2=timer()
total_train_time_model_2=print_train_time(start=train_time_start_m2,
                                          end=train_time_end_m2,
                                          device=device)

In [ ]:
model_2_results=eval_model(model=model_2,data_loader=test_dataloader,loss_fn=loss_fn,accuracy_fn=accuracy_fn)
model_2_results

In [ ]:
model_0_results

In [ ]:
import pandas as pd
compare_results=pd.DataFrame([model_0_results,
                              model_1_results,
                              model_2_results])
compare_results

In [ ]:
compare_results["training_time"]=[total_train_time_model_0,
                                  total_train_time_model_1,
                                  total_train_time_model_2]


In [ ]:
compare_results

In [ ]:
import matplotlib.pyplot as plt
compare_results.set_index("model_name")["model_accuracy"].plot(kind="barh")
plt.xlabel("accuracy %")
plt.ylabel("model")

In [ ]:
test_data

In [ ]:
def make_predictions(model:torch.nn.Module,
                     data:list,
                     device:torch.device=device):
  pred_probs=[]

  model.eval()
  with torch.inference_mode():
    for sample in data:

      pred_logits=model(sample.unsqueeze(dim=0).to(device))

      pred_probab=torch.softmax(pred_logits.squeeze(),dim=0)

      pred_probs.append(pred_probab.cpu())



  return torch.stack(pred_probs)


In [ ]:
import random
#random.seed(42)
test_samples=[]
test_labels=[]
for sample, label in random.sample(list(test_data),k=9):
   test_samples.append(sample)
   test_labels.append(label)

test_samples[0].shape

In [ ]:
pred_probs=make_predictions(model=model_2,data=test_samples)
pred_probs[:3]

In [ ]:
pred_classes=pred_probs.argmax(dim=1)
pred_classes

In [ ]:
plt.figure(figsize=(12,9))
nrows=3
ncolumns=3
for i,sample in enumerate(test_samples):

  plt.subplot(nrows,ncolumns,i+1)

  plt.imshow(sample.squeeze(),cmap="gray")

  pred_label=class_name[pred_classes[i]]

  truth_label=class_name[test_labels[i]]

  title_text=f"Pred:{pred_label} | Truth:{truth_label}"

  if pred_label==truth_label:
    plt.title(title_text,c="g")
  else:
    plt.title(title_text,c='r')

  plt.axis(False)

In [ ]:
!pip install torchmetrics

In [ ]:
from torchmetrics import ConfusionMatrix

In [ ]:
from tqdm.auto import tqdm
y_preds=[]
model_2.eval()
with torch.inference_mode():
  for X,y in tqdm(test_dataloader, desc="Making predictions...."):
    X,y=X.to(device),y.to(device)

    y_logit=model_2(X)

    y_pred=torch.softmax(y_logit.squeeze(),dim=0).argmax(dim=1)

    y_preds.append(y_pred.cpu())

#print(y_preds)
y_pred_tensor=torch.cat(y_preds)
y_pred_tensor[:10]

In [ ]:
!pip install mlxtend

In [ ]:
import mlxtend

In [ ]:
import torchmetrics
torchmetrics.__version__

In [ ]:
test_data.targets

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

confmat=ConfusionMatrix(num_classes=len(class_name),task="multiclass")
confmat_tensor=confmat(preds=y_pred_tensor,target=test_data.targets)

fig, ax=plot_confusion_matrix(conf_mat=confmat_tensor.numpy(),
                              class_names=class_name,
                              figsize=(10,7))

In [ ]:
confmat_tensor

In [ ]:
from pathlib import Path

MODEL_PATH=Path("model")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME="Computer_vision.pth"
MODEL_SAVE_PATH=MODEL_PATH/MODEL_NAME

torch.save(obj=model_2.state_dict(),f=MODEL_SAVE_PATH)

print("saving model to :",MODEL_SAVE_PATH)

In [ ]:
torch.manual_seed(42)
loaded_model_2=FashionMNISTModel2(input_shape=1,
                                  hidden_units=10,
                                  output_shape=10)

loaded_model_2.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

loaded_model_2.to(device)

In [ ]:
model_2_results

In [ ]:
torch.manual_seed(42)

loaded_model_2_results=eval_model(model=loaded_model_2,
                                  data_loader=test_dataloader,
                                  loss_fn=loss_fn,
                                  accuracy_fn=accuracy_fn)

loaded_model_2_results